# ETL Processes
Use this notebook to develop the ETL process for each of your tables before completing the `etl.py` file to load the whole datasets.

In [1]:
import os
import glob
import psycopg2

import pandas as pd
from sql_queries import create_table_queries

In [5]:
conn.close()

InterfaceError: cursor already closed

In [2]:
conn = psycopg2.connect("host=127.0.0.1 dbname=nubank user=postgres password=nubank")
cur = conn.cursor()

In [3]:
for query in create_table_queries:
        cur.execute(query)
        conn.commit()

In [4]:
#VERIFY THE DB TABLES
cur.execute("select relname from pg_class where relkind='r' and relname !~ '^(pg_|sql_)';")
print(cur.fetchall())

[('accounts_table',), ('city_table',), ('costumers_table',), ('country_table',), ('d_month_table',), ('d_time_table',), ('d_week_table',), ('d_year_table',), ('pix_movements_table',)]


In [6]:
def get_files(filepath):
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.csv'))
        for f in files :
            all_files.append(os.path.abspath(f))
    
    return all_files

# Process table data
In this first part, you'll perform ETL on the tables

In [7]:
table_files = get_files("tables/tables")

In [10]:
filepath = table_files

In [11]:
df = pd.read_csv(filepath)
df.head()

ValueError: Invalid file path or buffer object type: <class 'list'>

In [12]:
for file in filepath:
    df = pd.read_csv(file)
    df.head()
    print(df.isna().sum())
    

account_id             0
customer_id            0
created_at             0
status                 0
account_branch         0
account_check_digit    0
account_number         0
dtype: int64
city        0
state_id    0
city_id     0
dtype: int64
customer_id      0
first_name       0
last_name        0
customer_city    0
cpf              0
country_name     0
dtype: int64
country       0
country_id    0
dtype: int64
month_id        0
action_month    0
dtype: int64
time_id             0
action_timestamp    0
week_id             0
month_id            0
year_id             0
weekday_id          0
dtype: int64
weekday_id        0
action_weekday    0
dtype: int64
week_id        0
action_week    0
dtype: int64
year_id        0
action_year    0
dtype: int64
id                      0
account_id              0
in_or_out               0
pix_amount              0
pix_requested_at        0
pix_completed_at    12599
status                  0
dtype: int64
state         0
country_id    0
state_id      0
d

## #1: `songs` Table
#### Extract Data for Songs Table
- Select columns for song ID, title, artist ID, year, and duration
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `song_data`

In [10]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']]
song_data.head()

,song_id,title,artist_id,year,duration
0,SOMZWCG12A8C13C480,I Didn't Mean To,ARD7TVE1187B99BFB1,0,218.93179


In [11]:
# Save the dataframe to disk
tmp_df = "./tmp_dataframe.csv"
song_data.to_csv(tmp_df,index=False , header=False)
f = open(tmp_df, 'r')
    
#start = time.time()
cur.copy_from(f, 'song_table', sep=",")
#end = time.time()

In [12]:
song_data = df[['song_id', 'title', 'artist_id', 'year', 'duration']].values[0].tolist()
song_data

['SOMZWCG12A8C13C480', "I Didn't Mean To", 'ARD7TVE1187B99BFB1', 0, 218.93179]

#### Insert Record into Song Table
Implement the `song_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song into the `songs` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songs` table in the sparkify database.

In [13]:
cur.execute(song_table_insert, song_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

## #2: `artists` Table
#### Extract Data for Artists Table
- Select columns for artist ID, name, location, latitude, and longitude
- Use `df.values` to select just the values from the dataframe
- Index to select the first (only) record in the dataframe
- Convert the array to a list and set it to `artist_data`

In [14]:
artist_data = df[['artist_id', 'artist_name', 'artist_location', 'artist_latitude', 'artist_longitude']].values[0].tolist()
artist_data

['ARD7TVE1187B99BFB1', 'Casual', 'California - LA', nan, nan]

#### Insert Record into Artist Table
Implement the `artist_table_insert` query in `sql_queries.py` and run the cell below to insert a record for this song's artist into the `artists` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `artists` table in the sparkify database.

In [15]:
cur.execute(artist_table_insert, artist_data)
conn.commit()

Run `test.ipynb` to see if you've successfully added a record to this table.

# Process `log_data`
In this part, you'll perform ETL on the second dataset, `log_data`, to create the `time` and `users` dimensional tables, as well as the `songplays` fact table.

Let's perform ETL on a single log file and load a single record into each table.
- Use the `get_files` function provided above to get a list of all log JSON files in `data/log_data`
- Select the first log file in this list
- Read the log file and view the data

In [16]:
log_files = get_files("data/log_data")

In [17]:
filepath_log = log_files
filepath_log

['/home/workspace/data/log_data/2018/11/2018-11-30-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-11-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-23-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-24-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-25-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-20-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-15-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-01-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-03-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-13-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-10-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-28-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-26-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-04-events.json',
 '/home/workspace/data/log_data/2018/11/2018-11-14-events.json',
 '/home/workspace/data/lo

In [18]:
df_log = pd.read_json(filepath_log[0], lines=True)
for file1 in range(1,len(filepath_log)):
    df_log1 = pd.read_json(filepath_log[file1], lines=True)
    df_log = df_log.append(df_log1, ignore_index = True)
df = df_log
df_log.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8056 entries, 0 to 8055
Data columns (total 18 columns):
artist           6820 non-null object
auth             8056 non-null object
firstName        7770 non-null object
gender           7770 non-null object
itemInSession    8056 non-null int64
lastName         7770 non-null object
length           6820 non-null float64
level            8056 non-null object
location         7770 non-null object
method           8056 non-null object
page             8056 non-null object
registration     7770 non-null float64
sessionId        8056 non-null int64
song             6820 non-null object
status           8056 non-null int64
ts               8056 non-null int64
userAgent        7770 non-null object
userId           8056 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 1.1+ MB


## #3: `time` Table
#### Extract Data for Time Table
- Filter records by `NextSong` action
- Convert the `ts` timestamp column to datetime
  - Hint: the current timestamp is in milliseconds
- Extract the timestamp, hour, day, week of year, month, year, and weekday from the `ts` column and set `time_data` to a list containing these values in order
  - Hint: use pandas' [`dt` attribute](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.dt.html) to access easily datetimelike properties.
- Specify labels for these columns and set to `column_labels`
- Create a dataframe, `time_df,` containing the time data for this file by combining `column_labels` and `time_data` into a dictionary and converting this into a dataframe

In [19]:
df = df.query('page == "NextSong"')
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6820 entries, 0 to 8055
Data columns (total 18 columns):
artist           6820 non-null object
auth             6820 non-null object
firstName        6820 non-null object
gender           6820 non-null object
itemInSession    6820 non-null int64
lastName         6820 non-null object
length           6820 non-null float64
level            6820 non-null object
location         6820 non-null object
method           6820 non-null object
page             6820 non-null object
registration     6820 non-null float64
sessionId        6820 non-null int64
song             6820 non-null object
status           6820 non-null int64
ts               6820 non-null int64
userAgent        6820 non-null object
userId           6820 non-null object
dtypes: float64(2), int64(4), object(12)
memory usage: 1012.3+ KB


In [20]:
t = df['date'] = pd.to_datetime(df['ts'],unit='ms')

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [21]:
time_data = [t, df['date'].dt.hour, df['date'].dt.day, df['date'].dt.weekofyear, df['date'].dt.month, df['date'].dt.year, df['date'].dt.dayofweek]
column_labels = ['start_time', 'hour', 'day', 'week', 'month', 'year', 'weekday'] 

In [22]:
time_df = pd.DataFrame.from_dict(dict(zip(column_labels, time_data)))
time_df.head()

,start_time,hour,day,week,month,year,weekday
0,2018-11-30 00:22:07.796,0,30,48,11,2018,4
1,2018-11-30 01:08:41.796,1,30,48,11,2018,4
2,2018-11-30 01:12:48.796,1,30,48,11,2018,4
3,2018-11-30 01:17:05.796,1,30,48,11,2018,4
4,2018-11-30 01:20:56.796,1,30,48,11,2018,4


#### Insert Records into Time Table
Implement the `time_table_insert` query in `sql_queries.py` and run the cell below to insert records for the timestamps in this log file into the `time` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `time` table in the sparkify database.

In [23]:
for i, row in time_df.iterrows():
    cur.execute(time_table_insert, list(row))
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #4: `users` Table
#### Extract Data for Users Table
- Select columns for user ID, first name, last name, gender and level and set to `user_df`

In [24]:
user_df = df[['userId', 'firstName', 'lastName', 'gender', 'level']]
user_df["userId"] = pd.to_numeric(user_df["userId"])
user_df = user_df.drop_duplicates(subset=['userId'])
user_df.head(5)

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,userId,firstName,lastName,gender,level
0,91,Jayden,Bell,M,free
1,73,Jacob,Klein,M,paid
23,86,Aiden,Hess,M,free
30,24,Layla,Griffin,F,paid
40,26,Ryan,Smith,M,free


#### Insert Records into Users Table
Implement the `user_table_insert` query in `sql_queries.py` and run the cell below to insert records for the users in this log file into the `users` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `users` table in the sparkify database.

In [25]:
for i, row in user_df.iterrows():
    cur.execute(user_table_insert, row)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

## #5: `songplays` Table
#### Extract Data and Songplays Table
This one is a little more complicated since information from the songs table, artists table, and original log file are all needed for the `songplays` table. Since the log file does not specify an ID for either the song or the artist, you'll need to get the song ID and artist ID by querying the songs and artists tables to find matches based on song title, artist name, and song duration time.
- Implement the `song_select` query in `sql_queries.py` to find the song ID and artist ID based on the title, artist name, and duration of a song.
- Select the timestamp, user ID, level, song ID, artist ID, session ID, location, and user agent and set to `songplay_data`

#### Insert Records into Songplays Table
- Implement the `songplay_table_insert` query and run the cell below to insert records for the songplay actions in this log file into the `songplays` table. Remember to run `create_tables.py` before running the cell below to ensure you've created/resetted the `songplays` table in the sparkify database.

In [26]:
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

    # insert songplay record
    songplay_data = (row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()

Run `test.ipynb` to see if you've successfully added records to this table.

# Close Connection to Sparkify Database

In [99]:
conn.close()

# Implement `etl.py`
Use what you've completed in this notebook to implement `etl.py`.